In [34]:
import torch
#from datasets import load_dataset


#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self,path) :
        super().__init__()
        self.path = path 
        file = open(path,'r',encoding='utf-8')
        cases = file.readlines()
        self.cases = [i for i in  cases]
        self.length = len(self.cases)
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        data = self.cases[index]
    
        label=int(data[-2])
        content=data[:-2]

       
        return content,label

trainset=Dataset("./datasets/train.txt")
len(trainset)

24221

In [35]:
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('DPR')
from transformers import AutoModel

#加载预训练模型
dpr = AutoModel.from_pretrained('DPR')
dpr

Some weights of BertModel were not initialized from the model checkpoint at DPR and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(23283, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [5]:
def collate_fn(data):
    s= [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = tokenizer(s, return_tensors="pt",padding="max_length",max_length=300,truncation=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels




In [36]:
loader = torch.utils.data.DataLoader(dataset=trainset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

1513


(torch.Size([16, 300]),
 torch.Size([16, 300]),
 torch.Size([16, 300]),
 tensor([1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking arugment for argument index in method wrapper_index_select)

In [31]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = dpr(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])

        out = out.softmax(dim=1)

        return out


model = Model()

dpr=dpr.to(device)

device="cuda:0" if torch.cuda.is_available else "cpu"
from transformers import AdamW
from tqdm import trange,tqdm

#训练
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()
model.to(device)
model.train()
for j in range(3):
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in  tqdm(enumerate(loader),total=len(loader),colour='blue'):
        input_ids=input_ids.to(device)
        attention_mask=attention_mask.to(device)
        token_type_ids=token_type_ids.to(device)
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)

        labels =labels.to(device)

        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 1000 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            print(i, loss.item(), accuracy)

        if i == len(loader):
            break

  0%|▏                                                                                | 3/1513 [00:00<03:37,  6.93it/s]

0 0.6941920518875122 0.5625


 66%|███████████████████████████████████████████████████▋                          | 1003/1513 [01:14<00:37, 13.58it/s]

1000 0.45890504121780396 0.875


  0%|                                                                                 | 2/1513 [00:00<01:41, 14.85it/s]

...

  0%|                                                                                 | 2/1513 [00:00<02:02, 12.34it/s]

0 0.48952117562294006 0.8125


 66%|███████████████████████████████████████████████████▋                          | 1002/1513 [01:13<00:36, 13.91it/s]

1000 0.4271385073661804 0.875


100%|██████████████████████████████████████████████████████████████████████████████| 1513/1513 [01:51<00:00, 13.60it/s]


In [41]:
testset=Dataset("./datasets/test.txt")
def Test():
    model.eval()
    model.to("cpu")
    correct=0
    total=0
    
    loader_test=torch.utils.data.DataLoader(dataset=testset,
                                           batch_size=1,
                                           collate_fn=collate_fn,
                                           shuffle=True,
                                           drop_last=True)
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in  tqdm(enumerate(loader_test),total=len(loader_test),colour='green'):
        with torch.no_grad():
            out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)
            out=out.argmax(dim=1)
            correct+=(out==labels)
            total+=1
    print(correct/total)
Test()

100%|██████████████████████████████████████████████████████████████████████████████| 1498/1498 [05:02<00:00,  4.96it/s]

tensor([0.8425])


In [42]:
torch.save(model,"./model.pkl")

In [ ]:
fn=collate_fn)
    